In [2]:
from pprint import pprint
import json
import sys
sys.path.insert(0, '../genepattern-python/')
from gp import functions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%%HTML
<!--!AUTO_EXEC-->
<paper-material elevation="1" class="task-widget"><div class="task-widget-header item-header"><h2>Load JSON</h2></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="task-widget-inner" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="task-widget-content item-content"><div class="widget-info">No description.</div><form is="iron-form" class="task-widget-form"><div class="widget-form-panel"><div class="field-group field-required_args-group"><div class="item-header"><h3>Input</h3></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="field-group-content item-content"><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Simpli JSON to be Loaded" name="required_args" value="/Users/Jumis/tools/genepattern-python/functions.json" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">No description.</div></iron-collapse></div></iron-collapse></div></div><button class="form-submit-button-wrapper"><paper-button role="button" tabindex="0" animated="" aria-disabled="false" elevation="1" class="form-submit-button" raised="">submit<iron-icon icon="assessment"></iron-icon></paper-button></button></form></div></iron-collapse></paper-material>
<!--{"Load JSON":{"library_path":"","library_name":"simpli.default_tasks","function_name":"link_json","description":"No description.","required_args":[{"name":"filepath","value":"/Users/Jumis/tools/genepattern-python/functions.json","label":"Simpli JSON to be Loaded","description":"No description."}],"default_args":[],"optional_args":[],"returns":[]}}-->

In [ ]:
functions.login('https://genepattern.broadinstitute.org/gp', 'Simpli', 'Simpli')
functions.get_tasks()

t = functions.GPTask(functions.SERVER, 'PreprocessDataset')
t.param_load()
name = t.get_name()

required_args = []
default_args = [{'name': 'task_name', 'value': name}]
optional_args = []
other_information = {}

task_entry = {
    "function_path": 'gp.gp_functions.execute',
    "label": name,
    "description": t.get_description(),
    "required_args": required_args,
    "default_args": default_args,
    "optional_args": optional_args,
    "returns": [{'label': 'Job', 
                 'description': 'ONLY FOR TESTING: RETURNS FIELD WILL BE REMOVED'}],
    "other_information": other_information,
}

tasks = []

task_json = {
    "library_path": "Jumis/tools/genepattern-python",
    "tasks": tasks,
}

tasks.append(
    {
        "function_path": 'gp.gp_functions.execute',
        "label": 'Login GenePattern (Broad Institute)',
        "description": 'Login into the GenePattern server at the Broad Institure.',
        "required_args":
        [            
            {
                'name': 'user_name',
                'label': 'User name',
                "description": 'GenePattern user name.',
            },
            {
                'name': 'passcode',
                'label': 'Passcode',
                "description": 'GenePattern passcode.',
            },        
        ],
        "default_args":
        [
            {
                'name': 'server_name',
                'value': 'https://genepattern.broadinstitute.org/gp',
            },
        ],
    }
            )

ps = t.get_parameters()
for p in ps:
    a = {}
    description = p.get_description()
    default_value = p.get_default_value()
    
    a['name'] = p.get_name()
    a['description'] = description
    if default_value:
        a['value'] = default_value
    
    type_ = p.get_type()
    if p.is_choice_param():
        choices = [d['value'] for d in p.get_choices()]
        a['description']: '{} (default={}) (choices={}) (type={})'.format(description, default_value, choices, type_,)
    else:
        a['description']: '{} (default={}) (type={})'.format(description, default_value, type_,)
    
    if default_value:
        optional_args.append(a)
    else:
        required_args.append(a)

other_information['lsid'] = t.get_lsid()
other_information['version'] = t.get_version()

tasks.append(task_entry)

pprint(task_json)
with open('/Users/kateme/Jumis/tools/genepattern-python/functions.json', 'w') as f:
    json.dump(task_json, f, indent=4)

js = functions.make_task_spec('PreprocessDataset')
# js.set_parameter('input.filename',
#                  'https://software.broadinstitute.org/cancer/software/genepattern/data/all_aml/all_aml_test.gct')
js.set_parameter('input.filename',
                 '/Users/kateme/Jumis/data/animals/animals.gct')
j = functions.submit_job(js)

In [3]:
print(j.is_finished())
for f in j.get_output_files():
    print(f.read())

True
Filtered all genes.



In [4]:
# import sys
# sys.path.insert(0, '')
# import simpli
# m = simpli.Manager(True)
# m.print_tasks_as_json()